# Team              : CTRL C CTRL V
# Track Company     : Track-1 Cactus Communications
# Track Name        : Paperpal - Future of Academic Writing
# Chec Type         : Content Centric Checks
# Check Implemented : Publication Integrity

### Salient Features: 

1. Typo errors are flagged and checked, by converting a pdf and preprocessing data using series of  preprocessors, autocorrects and spell checkers.
2. Plagiarism Checker uses web scraping from Wikipedia, assumed to be the authentic source, converted to a corpus, passed through a series of preprocessors and uses state-of-the art doc2Vec model. The model is trained and evaluated for, against our pdf/docx, uses co-sine similarity and produces results in decreasing sorted order of similarity.

# Typo

In [1]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.1 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [5]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=4d01c356fe5724e416e50c2972f9a3a3ecfe0ac7b7dff19773157c5a430c2ec3
  Stored in directory: /root/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect


In [6]:
import PyPDF2
import io
import random

pdfReader = PyPDF2.PdfReader("/content/Malicious_URL_detection_using_multi-layer_filtering_model.pdf")

num_pages = len(pdfReader.pages)
count = 0
text = ""
  #The while loop will read each page.
while count < num_pages:
  pageObj = pdfReader.pages[count]
  count +=1
  text += pageObj.extract_text()

import re

words = re.findall(r"[\s\w]", text)
words = "".join(words)
words = words.lower()
for i in range(len(words)):
  if words[i].isdigit():
    words = words[:i]+' '+words[i+1:]

words = words.split()
for i in words:
  if len(i)<3:
    words.remove(i)

stop = nltk.corpus.stopwords.words('english')
for i in stop:
  while i in words:
    words.remove(i)

words = ' '.join(words)

x = words.split()
tokens = []
for i in x:
  if len(i)>3:
    tokens.append(i)

tags = nltk.pos_tag(tokens)

from autocorrect import Speller
spell = Speller()
for i in range(len(tokens)):
  if tags[i][1] not in ['NN', 'JJ', 'NNP']:
    tokens[i] = spell(tokens[i])

print(tokens)

['ieee', 'malicious', 'detection', 'using', 'multilayer', 'filtering', 'model', 'rajesh', 'kumar', 'xiaosong', 'zhang', 'hussain', 'ahmad', 'tariq', 'riaz', 'ullah', 'khan', 'school', 'computer', 'science', 'engineering', 'university', 'electro', 'science', 'technology', 'china', 'email', 'rajakumarlohanogmailcom', 'rerukhanoutlookcom', 'abstract', 'malicious', 'urls', 'harmful', 'every', 'aspect', 'computer', 'users', 'detecting', 'malicious', 'important', 'currently', 'detection', 'malicious', 'webpages', 'techniques', 'includes', 'blacklist', 'whitelist', 'methodology', 'machine', 'learning', 'classification', 'algorithms', 'used', 'however', 'blacklist', 'whitelist', 'technology', 'useless', 'particular', 'list', 'paper', 'propose', 'multilayer', 'model', 'detecting', 'malicious', 'filter', 'directly', 'determine', 'training', 'threshold', 'layer', 'filter', 'reaches', 'threshold', 'otherwise', 'filter', 'also', 'used', 'example', 'verify', 'model', 'improve', 'accuracy', 'detectio

# Plag Check

In [7]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=af56f9fe6bfe6308993cc107a5ca83f8c5e5d7ba10fa77c2e222f8ab1f4122b7
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [8]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import wikipedia

wiki_pages = []
for i in tokens:
  try:
    wiki_pages.append(wikipedia.page(i))
  except:
    continue
print(wiki_pages)

model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=2, epochs=80)
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(text)]
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=80)

data = word_tokenize(' '.join(tokens))
token_vector = model.infer_vector(data)

wiki_dict = {}
for i in wiki_pages:
  data = word_tokenize(i.content)
  wiki_dict[i.url] = model.infer_vector(data)

def cosine_sim(a, b):
  sum = 0
  sum_a = 0
  sum_b = 0
  for i in range(len(a)):
    sum_a += a[i]**2
    sum_b += b[i]**2
    sum += (a[i]*b[i])

  return sum/((sum_a**(1/2))*(sum_b**(1/2)))

cos_vals = {}
for i in wiki_pages:
  cos_vals[i.url] = cosine_sim(token_vector, wiki_dict[i.url])

print('The descending order of similarity index with different websites is:')
ranked = sorted(cos_vals, key=lambda x:cos_vals[x], reverse=True)
for i in ranked:
  if cos_vals[i]>0.3:
    print(i, cos_vals[i])

/usr/local/lib/python3.8/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.8/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[<WikipediaPage 'Institute of Electrical and Electronics Engineers'>, <WikipediaPage 'Multiplayer video game'>, <WikipediaPage 'Rajesh'>, <WikipediaPage 'Kumar'>, <WikipediaPage 'Gao Xiaosong'>, <WikipediaPage 'Hussein'>, <WikipediaPage 'Ahmad'>, <WikipediaPage 'Tariq'>, <WikipediaPage 'Riaz'>, <WikipediaPage 'Ulla'>, <WikipediaPage 'School'>, <WikipediaPage 'Engineering'>, <WikipediaPage 'University'>, <WikipediaPage 'Electricity'>, <WikipediaPage 'Technology'>, <WikipediaPage 'Mail'>, <WikipediaPage 'URL'>, <WikipediaPage 'Harmful'>, <WikipediaPage 'Expect'>, <WikipediaPage 'Music Detected'>, <WikipediaPage 'Importance'>, <WikipediaPage 'Web page'>, <WikipediaPage 'Include directive'>, <WikipediaPage 'Blacklisting'>, <WikipediaPage 'Whitelist'>, <WikipediaPage 'Methodology'>, <WikipediaPage 'Machine'>, <WikipediaPage 'Classification'>, <WikipediaPage 'Algorithm'>, <WikipediaPage 'Blacklisting'>, <WikipediaPage 'Whitelist'>, <WikipediaPage 'Technology'>, <WikipediaPage 'Copula (lingui